In [1]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

Packages installed.


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [4]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [5]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [6]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
1,M2A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
2,M3A\n,North York\n,Parkwoods\n,43.75245,-79.32991
3,M4A\n,North York\n,Victoria Village\n,43.73057,-79.31306
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.65512,-79.36264
...,...,...,...,...,...
174,M4Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
175,M5Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
176,M6Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
177,M7Z\n,Not assigned\n,Not assigned\n,43.64869,-79.38544


In [7]:
neighbors.head(40)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
1,M2A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
2,M3A\n,North York\n,Parkwoods\n,43.75245,-79.32991
3,M4A\n,North York\n,Victoria Village\n,43.73057,-79.31306
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.65512,-79.36264
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.72327,-79.45042
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.66253,-79.39188
7,M8A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
8,M9A\n,Etobicoke\n,"Islington Avenue, Humber Valley Village\n",43.66263,-79.52831
9,M1B\n,Scarborough\n,"Malvern, Rouge\n",43.81139,-79.19662


In [8]:
VERSION = '20180605' # Foursquare API version

neighborhood_name = neighbors.loc[2, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[2, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

CLIENT_ID = '1PRXICNH4QM2MIH4LF4QIAJHMXQOQWTAWCQCXW4LC3L2WDUF' # your Foursquare ID
CLIENT_SECRET = 'D3RATHV1LMTLD1BM00QXYIZBNUOKP5MQBPCHDRSSLW3RIAOS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
     CLIENT_ID, 
     CLIENT_SECRET, 
     VERSION, 
     43.75245000000007, 
     -79.32990999999998, 
     radius, 
     LIMIT)

results = requests.get(url).json()
results

Latitude and longitude values of Parkwoods
 are 43.75245000000007, -79.32990999999998.
Your credentails:
CLIENT_ID: 1PRXICNH4QM2MIH4LF4QIAJHMXQOQWTAWCQCXW4LC3L2WDUF
CLIENT_SECRET:D3RATHV1LMTLD1BM00QXYIZBNUOKP5MQBPCHDRSSLW3RIAOS


{'meta': {'code': 200, 'requestId': '60401f816a66e33d37d93a67'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.75695000450007,
    'lng': -79.32369182386579},
   'sw': {'lat': 43.747949995500065, 'lng': -79.33612817613418}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLng

In [9]:
# From the Foursquare lab in the previous module, we know that all the information is in the items key. 
# Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [12]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

Not assigned

Not assigned

Parkwoods

Victoria Village

Regent Park, Harbourfront

Lawrence Manor, Lawrence Heights

Queen's Park, Ontario Provincial Government

Not assigned

Islington Avenue, Humber Valley Village

Malvern, Rouge

Not assigned

Don Mills

Parkview Hill, Woodbine Gardens

Garden District, Ryerson

Glencairn

Not assigned

Not assigned

West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale

Rouge Hill, Port Union, Highland Creek

Not assigned

Don Mills

Woodbine Heights

St. James Town

Humewood-Cedarvale

Not assigned

Not assigned

Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Guildwood, Morningside, West Hill

Not assigned

Not assigned

The Beaches

Berczy Park

Caledonia-Fairbanks

Not assigned

Not assigned

Not assigned

Woburn

Not assigned

Not assigned

Leaside

Central Bay Street

Christie

Not assigned

Not assigned

Not assigned

Cedarbrae

Hillcrest Village

Bathurst Manor, Wilson Heights, Downsview North

Thorncliffe P

In [14]:
# check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(358, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Not assigned\n,43.64869,-79.38544,Brookbanks Park,43.751976,-79.332140,Park
1,Not assigned\n,43.64869,-79.38544,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Not assigned\n,43.64869,-79.38544,Brookbanks Park,43.751976,-79.332140,Park
3,Not assigned\n,43.64869,-79.38544,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Parkwoods\n,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park


In [15]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt\n,2,2,2,2,2,2
"Alderwood, Long Branch\n",2,2,2,2,2,2
"Bathurst Manor, Wilson Heights, Downsview North\n",2,2,2,2,2,2
Bayview Village\n,2,2,2,2,2,2
"Bedford Park, Lawrence Manor East\n",2,2,2,2,2,2
...,...,...,...,...,...,...
"Willowdale, Willowdale West\n",2,2,2,2,2,2
Woburn\n,2,2,2,2,2,2
Woodbine Heights\n,2,2,2,2,2,2


In [16]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 2 uniques categories.


In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Food & Drink Shop,Park
0,Not assigned\n,0,1
1,Not assigned\n,1,0
2,Not assigned\n,0,1
3,Not assigned\n,1,0
4,Parkwoods\n,0,1


In [18]:
# examine the new dataframe size.

toronto_onehot.shape

(358, 3)

In [19]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Food & Drink Shop,Park
0,Agincourt\n,0.5,0.5
1,"Alderwood, Long Branch\n",0.5,0.5
2,"Bathurst Manor, Wilson Heights, Downsview North\n",0.5,0.5
3,Bayview Village\n,0.5,0.5
4,"Bedford Park, Lawrence Manor East\n",0.5,0.5
...,...,...,...
95,"Willowdale, Willowdale West\n",0.5,0.5
96,Woburn\n,0.5,0.5
97,Woodbine Heights\n,0.5,0.5
98,York Mills West\n,0.5,0.5


In [20]:
# confirm the new size

toronto_grouped.shape

(100, 3)

In [21]:
# Now print each neighborhood along with the top 5 most common venues

num_top_venues = 5


for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Alderwood, Long Branch
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Bathurst Manor, Wilson Heights, Downsview North
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Bayview Village
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Bedford Park, Lawrence Manor East
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Berczy Park
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Birch Cliff, Cliffside West
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Brockton, Parkdale Village, Exhibition Place
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Business reply mail Processing Centre, South

               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Rouge Hill, Port Union, Highland Creek
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Runnymede, Swansea
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Runnymede, The Junction North
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Scarborough Village
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----St. James Town
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----St. James Town, Cabbagetown
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Steeles 

In [22]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, 1:], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt\n,Park,Park,Park
1,"Alderwood, Long Branch\n",Park,Park,Park
2,"Bathurst Manor, Wilson Heights, Downsview North\n",Park,Park,Park
3,Bayview Village\n,Park,Park,Park
4,"Bedford Park, Lawrence Manor East\n",Park,Park,Park


In [24]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:9: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [25]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544,0,Park,Park,Park
1,M2A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544,0,Park,Park,Park
2,M3A\n,North York\n,Parkwoods\n,43.75245,-79.32991,0,Park,Park,Park
3,M4A\n,North York\n,Victoria Village\n,43.73057,-79.31306,0,Park,Park,Park
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.65512,-79.36264,0,Park,Park,Park


In [26]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[43.75245000000007, -79.32990999999998], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Not assigned\n,0,Park,Park,Park
1,Not assigned\n,0,Park,Park,Park
2,North York\n,0,Park,Park,Park
3,North York\n,0,Park,Park,Park
4,Downtown Toronto\n,0,Park,Park,Park
...,...,...,...,...,...
174,Not assigned\n,0,Park,Park,Park
175,Not assigned\n,0,Park,Park,Park
176,Not assigned\n,0,Park,Park,Park
177,Not assigned\n,0,Park,Park,Park


In [28]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue


In [29]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue


In [30]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue


In [31]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
